In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, multiply, concatenate, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
import pandas as pd

In [3]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [5]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [6]:
print(des.shape)
print(body.shape)
print(y.shape)
print(y_1.shape)

(828, 13)
(828, 34)
(828,)
(828, 1)


In [7]:
hid_layer = [100, 100, 50]
print(hid_layer[-1])

50


In [8]:
_, des_col = des.shape
_, body_col = body.shape

In [9]:
from keras import backend as K
from keras.layers import Layer

def recurent_matrix(des_shape, body_shape): # circulant matrix 만들어주는 함수

    shape = des_shape.get_shape().as_list() # shape 저장.
    des_set = [] 
    des0 = K.expand_dims(des_shape, 2) # expand_dims: 차원을 늘려주는 함수
    des_set.append(des0)

    body_set = []
    body0 = K.expand_dims(body_shape, 2)
    body_set.append(body0)
    
    
    for i in range(1,shape[1]): # 행렬 생성 shape[1]

        pre=des0[:,(shape[1]-i):,:]
        host=des0[:,0:(shape[1]-i),:]
        des1=tf.concat([pre,host],1)
        des_set.append(des0)

        pre=body0[:,(shape[1]-i):,:]
        host=body0[:,0:(shape[1]-i),:]
        body1=tf.concat([pre,host],1)
        body_set.append(body1)

    vector_des=K.concatenate(des_set) # 3차원 concatenate
    vector_body=K.concatenate(body_set)
    
    multiple_vector1 = K.batch_dot(vector_body, des0)
    multiple_vector2 = K.batch_dot(vector_des, body0)
    
    sum_vector = multiple_vector1 + multiple_vector2

    return sum_vector

In [19]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

def circulant(shape):
    matrix_shape = shape.get_shape().as_list()
    circulant_set = []
    circulant_ = K.expand_dims(shape, 2)
    circulant_set.append(circulant_)

    for i in range(1, matrix_shape[1]):
        pre=circulant_[:,(matrix_shape[1]-i):,:]
        host=circulant_[:,0:(matrix_shape[1]-i),:]
        circulant_1=tf.concat([pre,host],1)
        circulant_set.append(circulant_1)

    vector_ = K.concatenate(circulant_set)

    return vector_

class Circulant_layer(Layer):
    def __init__(self, output_dim, **kwargs):
        #self.output_dim = output_dim
        self.output_dim = output_dim
        super(Circulant_layer, self).__init__(**kwargs)
   
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[0][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Circulant_layer, self).build(input_shape) 

    def call(self, x):
        assert isinstance(x, list)
        des, body = x
        print(self.kernel.shape)
        des = K.dot(des, self.kernel) #, self.kernel
        body = K.dot(body, self.kernel)
        
        multiple_vector1 = K.batch_dot(circulant(body), des)
        multiple_vector2 = K.batch_dot(circulant(des), body)
        
        sum_vector = multiple_vector1 + multiple_vector2
        sum_vector_1 = K.dot(sum_vector, self.kernel)
        
        return sum_vector

In [20]:
class early_Concat:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi(self):
        # des modality
        des_input = Input(shape = (self.des_shape,)) # input = 17, des_shape = 17
        body_input = Input(shape = (self.body_shape,)) # input = 30,body_shape = 30

        des_reshape = Dense(50,)(des_input)
        body_reshape = Dense(50,)(body_input)
        
        fusion = Circulant_layer(self.hid_layer[2])([des_reshape, body_reshape])
        
        tensor_flat = Flatten()(fusion)
        #x = Dense(self.des_shape + self.body_shape , activation = 'relu')(concat)
        #x = BatchNormalization()(x)
        
        x = Dense(hid_layer[0] , activation = 'relu')(tensor_flat)
        x = BatchNormalization()(x)

        x = Dense(hid_layer[1] , activation = 'relu')(x)
        x = BatchNormalization()(x)

        x = Dense(hid_layer[2] , activation = 'relu')(x)
        x = BatchNormalization()(x)
        
        x = Dense(self.output_layer , activation = 'sigmoid')(x)
        
        early_fusion = keras.models.Model(inputs = [des_input, body_input], outputs = [x])

        return early_fusion

In [21]:
final_model = early_Concat(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi()
print(network)

(50, 50)


In [38]:
network.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 50)           700         input_9[0][0]                    
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 50)           1750        input_10[0][0]                   
____________________________________________________________________________________________

In [53]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 5, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

5

In [40]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [41]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
                
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_sm_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_cw_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_smcw_y[index] = y_pred[i]
            
        return self.pd_smcw_y
    

In [42]:
final_model = early_Concat(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi()
print(network)

f_y, f_sm_y, f_cw_y,f_smcw_y = [], [], [], []
output_fusion = train_model(X, y, network, dict_y, dict_y, dict_y, dict_y)

# sum model training
f_y = output_fusion.train_based()
#f_sm_y = output_fusion.train_smote()
#f_cw_y = output_fusion.train_weight()
#f_smcw_y = output_fusion.train_sm_weight()

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 4ms/step - loss: 0.8152 - accuracy: 0.5208
Epoch 2/50
24/24 [==============================] - 0s 4ms/step - loss: 0.5851 - accuracy: 0.7087
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 0.5154 - accuracy: 0.8107
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4303 - accuracy: 0.8846
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 0.3994 - accuracy: 0.8899
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 0.3452 - accuracy: 0.9074
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - lo

24/24 [==============================] - 0s 4ms/step - loss: 0.0312 - accuracy: 0.9893
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0152 - accuracy: 0.9973
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0145 - accuracy: 0.9960
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0207 - accuracy: 0.9893
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0261 - accuracy: 0.9960
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0266 - accuracy: 0.9946
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0233 - accuracy: 0.9919
Epoch 9/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0193 - accuracy: 0.9906
Epoch 10/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0235 - accuracy: 0.9946
Epoch 11/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0153 - accuracy: 0.9946
Epoch 12/50
24/24 [==========

24/24 [==============================] - 0s 4ms/step - loss: 0.0187 - accuracy: 0.9933
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0202 - accuracy: 0.9919
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0226 - accuracy: 0.9906
Epoch 9/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0163 - accuracy: 0.9946
Epoch 10/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0112 - accuracy: 0.9960
Epoch 11/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 12/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0082 - accuracy: 0.9987
Epoch 13/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0094 - accuracy: 0.9960
Epoch 14/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9946
Epoch 15/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0064 - accuracy: 0.9987
Epoch 16/50
24/24 [======

24/24 [==============================] - 0s 4ms/step - loss: 0.0238 - accuracy: 0.9946
Epoch 36/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 37/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 38/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0056 - accuracy: 0.9987
Epoch 39/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0181 - accuracy: 0.9960
Epoch 40/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0357 - accuracy: 0.9919
Epoch 41/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0469 - accuracy: 0.9893
Epoch 42/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0866 - accuracy: 0.9732
Epoch 43/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0865 - accuracy: 0.9705
Epoch 44/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0822 - accuracy: 0.9758
Epoch 45/50
24/24 [===

In [43]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        print(confusion.ravel())
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy) #pb
        print("Precision(pb) :",mean_precision )
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb

        print("MCC :", mcc)
        print("G-mean :", g_mean)


        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 

        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [59]:
f_y_result = Evaluation(predict_change(f_y), y)
#f_sm_y_result = Evaluation(f_sm_y, y)
#f_cw_y_result = Evaluation(predict_change(f_cw_y), y)
#f_smcw_y_result = Evaluation(f_smcw_y, y)


print("#MCF model 결과 \n")
print("\n-----based model-----\n")
print(f_y_result.matrix())

print("\n-----smote model-----\n")
#print(f_sm_y_result.matrix())

print("\n-----class weight model-----\n")
#print(f_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(f_smcw_y_result.matrix())

ValueError: cannot reshape array of size 0 into shape (0,newaxis)

In [61]:
fusion_y_set = []

for i in range(20):
    final_model = early_Concat(des_col, body_col, hid_layer, 1) # input type = integer
    network = final_model.Multi()
    print(network)

    f_y, f_sm_y, f_cw_y,f_smcw_y = [], [], [], []
    output_fusion = train_model(X, y, network, dict_y, dict_y, dict_y, dict_y)

    # sum model training
    f_y = output_fusion.train_based()
    #f_sm_y = output_fusion.train_smote()a
    #f_cw_y = output_fusion.train_weight()
    #f_smcw_y = output_fusion.train_sm_weight()
    
    f_y_result = Evaluation(predict_change(f_y), y_1)
    #f_sm_y_result = Evaluation(f_sm_y, y)
    #f_cw_y_result = Evaluation(predict_change(f_cw_y), y)
    #f_smcw_y_result = Evaluation(predict_change(f_smcw_y), y)
    
    fusion_y_set = np.append(fusion_y_set, f_y_result.matrix())
    print(len(fusion_y_set))
    
fusion_y_np = np.array(fusion_y_set)

fusion_y_np = fusion_y_np.reshape(-1,15)
fusion_y_np.shape

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Epoch 1/50
21/21 [==============================] - 0s 4ms/step - loss: 0.7409 - accuracy: 0.5801
Epoch 2/50
21/21 [==============================] - 0s 4ms/step - loss: 0.5921 - accuracy: 0.7251
Epoch 3/50
21/21 [==============================] - 0s 5ms/step - loss: 0.5112 - accuracy: 0.8051
Epoch 4/50
21/21 [=======

21/21 [==============================] - 0s 8ms/step - loss: 0.0202 - accuracy: 0.9924
Epoch 45/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0356 - accuracy: 0.9909
Epoch 46/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0334 - accuracy: 0.9894
Epoch 47/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0192 - accuracy: 0.9940
Epoch 48/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0196 - accuracy: 0.9909
Epoch 49/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0195 - accuracy: 0.9940
Epoch 50/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0262 - accuracy: 0.9894
TEST: [  4  14  17  28  29  35  38  41  44  45  47  48  51  55  61  78  81  88
  90  92  95 108 109 114 116 124 127 128 142 148 151 155 159 161 163 168
 171 174 179 181 191 203 206 207 213 216 217 220 229 235 245 247 253 254
 259 265 267 274 276 286 287 288 292 297 299 305 313 315 333 343 348 352
 362 365 369 372 376 

Epoch 13/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0245 - accuracy: 0.9955
Epoch 14/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0109 - accuracy: 0.9925
Epoch 15/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0082 - accuracy: 0.9985
Epoch 16/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0132 - accuracy: 0.9955
Epoch 17/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0099 - accuracy: 0.9970
Epoch 18/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0099 - accuracy: 0.9970
Epoch 19/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0127 - accuracy: 0.9970
Epoch 20/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0082 - accuracy: 0.9955
Epoch 21/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0047 - accuracy: 0.9985
Epoch 22/50
21/21 [==============================] - 0s 8ms/step - loss: 0.0062 - accuracy: 0.9985
Epoch 23/5

21/21 [==============================] - 0s 6ms/step - loss: 0.1331 - accuracy: 0.9698
Epoch 2/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0503 - accuracy: 0.9849
Epoch 3/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0341 - accuracy: 0.9924
Epoch 4/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0251 - accuracy: 0.9924
Epoch 5/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0310 - accuracy: 0.9834
Epoch 6/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0187 - accuracy: 0.9940
Epoch 7/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0176 - accuracy: 0.9940
Epoch 8/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0240 - accuracy: 0.9879
Epoch 9/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0345 - accuracy: 0.9864
Epoch 10/50
21/21 [==============================] - 0s 8ms/step - loss: 0.0474 - accuracy: 0.9879
Epoch 11/50
21/21 [===========

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Epoch 1/50
21/21 [==============================] - 0s 7ms/step - loss: 0.7210 - accuracy: 0.5785
Epoch 2/50
21/21 [==============================] - 0s 7ms/step - loss: 0.6164 - accuracy: 0.6949
Epoch 3/50
21/21 [==============================] - 0s 7ms/step - loss: 0.5236 - accuracy: 0.7885
Epoch 4/50
21/21 [=======

...

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Epoch 1/50
21/21 [==============================] - 0s 7ms/step - loss: 0.7500 - accuracy: 0.5846
Epoch 2/50
21/21 [==============================] - 0s 7ms/step - loss: 0.5952 - accuracy: 0.7221
Epoch 3/50
21/21 [==============================] - 0s 6ms/step - loss: 0.5161 - accuracy: 0.7961
Epoch 4/50
21/21 [=======

21/21 [==============================] - 0s 7ms/step - loss: 0.0206 - accuracy: 0.9924
Epoch 46/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0177 - accuracy: 0.9940
Epoch 47/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0186 - accuracy: 0.9970
Epoch 48/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0219 - accuracy: 0.9924
Epoch 49/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0153 - accuracy: 0.9955
Epoch 50/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0189 - accuracy: 0.9940
TEST: [  4  14  17  28  29  35  38  41  44  45  47  48  51  55  61  78  81  88
  90  92  95 108 109 114 116 124 127 128 142 148 151 155 159 161 163 168
 171 174 179 181 191 203 206 207 213 216 217 220 229 235 245 247 253 254
 259 265 267 274 276 286 287 288 292 297 299 305 313 315 333 343 348 352
 362 365 369 372 376 382 387 390 392 393 394 398 400 404 417 424 427 430
 435 445 448 453 454 457 478 479 480 482 494 50

21/21 [==============================] - 0s 6ms/step - loss: 0.0684 - accuracy: 0.9789
Epoch 34/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0672 - accuracy: 0.9834
Epoch 35/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0622 - accuracy: 0.9789
Epoch 36/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0532 - accuracy: 0.9849
Epoch 37/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0585 - accuracy: 0.9819
Epoch 38/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0552 - accuracy: 0.9819
Epoch 39/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0424 - accuracy: 0.9864
Epoch 40/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0387 - accuracy: 0.9924
Epoch 41/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0394 - accuracy: 0.9879
Epoch 42/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0478 - accuracy: 0.9819
Epoch 43/50
21/21 [===

21/21 [==============================] - 0s 7ms/step - loss: 0.0114 - accuracy: 0.9970
Epoch 29/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0227 - accuracy: 0.9925
Epoch 30/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0288 - accuracy: 0.9925
Epoch 31/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0106 - accuracy: 0.9970
Epoch 32/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0228 - accuracy: 0.9910
Epoch 33/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0130 - accuracy: 0.9940
Epoch 34/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0176 - accuracy: 0.9925
Epoch 35/50
21/21 [==============================] - 0s 7ms/step - loss: 0.0082 - accuracy: 0.9985
Epoch 36/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0074 - accuracy: 0.9985
Epoch 37/50
21/21 [==============================] - 0s 6ms/step - loss: 0.0069 - accuracy: 0.9985
Epoch 38/50
21/21 [===

(20, 15)

In [47]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [62]:
result(fusion_y_np)

공통 
AUC :
 [평균 : 0.92488 ] [표준편차 : 0.01413 ]
AUPRC :
 [평균 : 0.99136 ] [표준편차 : 0.00224 ]
Optimized precision :
 [평균 : 0.32329 ] [표준편차 : 0.02271 ]

function 사용
Accuracy :
 [평균 : 0.94052 ] [표준편차 : 0.00722 ]
Precision(pb) :
 [평균 : 0.98754 ] [표준편차 : 0.00230 ]
Recall(pb) :
 [평균 : 0.57767 ] [표준편차 : 0.00898 ]
F1 score(pb) :
 [평균 : 0.72890 ] [표준편차 : 0.00722 ]
MCC :
 [평균 : 0.57938 ] [표준편차 : 0.05083 ]
G-mean :
 [평균 : 0.71759 ] [표준편차 : 0.03176 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.94052 ] [표준편차 : 0.00722 ]
Precision :
 [평균 : 0.95665 ] [표준편차 : 0.00405 ]
Recall :
 [평균 : 0.97933 ] [표준편차 : 0.00550 ]
F1 score :
 [평균 : 0.96785 ] [표준편차 : 0.00393 ]
MCC :
 [평균 : 0.59023 ] [표준편차 : 0.04769 ]
G-mean :
 [평균 : 0.71759 ] [표준편차 : 0.03176 ]
